<a href="https://colab.research.google.com/github/Elshraby/GradProj2/blob/main/12_May_PredictionsOptuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load a trained VAWT model from disk

import torch
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import torch.nn as nn

class StandardVAWTNet(nn.Module):
    def __init__(self, num_feature_size, airfoil_vocab_size, emb_size=4, hidden_size=128, dropout=0.2):
        super(StandardVAWTNet, self).__init__()
        self.airfoil_embedding = nn.Embedding(airfoil_vocab_size, emb_size)
        input_size = num_feature_size + emb_size
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, num_features, airfoil_type):
        emb = self.airfoil_embedding(airfoil_type)
        combined = torch.cat([num_features, emb], dim=1)
        return self.model(combined)

# Load model metadata
with open("best_model_metadata.json", "r") as f:
    metadata = json.load(f)

# Instantiate the model
model = StandardVAWTNet(
    num_feature_size=metadata["input_dim"],
    airfoil_vocab_size=metadata["airfoil_vocab_size"],
    hidden_size=metadata["hidden_size"],
    dropout=metadata["dropout"]
)

# Load the trained weights
model.load_state_dict(torch.load("best_vawt_model.pth"))
model.eval()
print("✅ Model loaded successfully.")

# Optional prediction function
def predict(model, num_features, airfoil_code):
    with torch.no_grad():
        num_tensor = torch.tensor([num_features], dtype=torch.float32)
        airfoil_tensor = torch.tensor([airfoil_code], dtype=torch.int64)
        pred = model(num_tensor, airfoil_tensor)
        return pred.item()

# Example usage
# Replace with real values from your dataset:
sample_input = [3, 0.2, 0.5, 0.6, 0, 7.0, 1.2]  # example numerical features
sample_airfoil_code = 0  # you should map this from category encoding
predicted_cp = predict(model, sample_input, sample_airfoil_code)
print(f"Predicted Cp: {predicted_cp:.4f}")

✅ Model loaded successfully.
Predicted Cp: 0.1423
